In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip3 install vncorenlp
!test -d VnCoreNLP || git clone --depth=1 https://github.com/vncorenlp/VnCoreNLP

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 5.7 MB/s 
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645951 sha256=8c71aeb48400333e1cc74c5c90349370b2aaf9b2dffaa55ada4c735a6ae3d987
  Stored in directory: /root/.cache/pip/wheels/0c/d8/f2/d28d97379b4f6479bf51247c8dfd57fa00932fa7a74b6aab29
Successfully built vncorenlp
Cloning into 'VnCoreNLP'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 54 (delta 1), reused 40 (delta 0), pack-reused 0
Unpacking objects: 100% (54/54), done.


In [ ]:
import os
import re
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
from vncorenlp import VnCoreNLP
import itertools
from more_itertools import pairwise

In [ ]:
viquad_dataset = '/content/drive/MyDrive/Colab Notebooks/MRC - VLSP/Dataset/ViQuADv1.1/test_ViQuAD.json'

with open(viquad_dataset) as viquad_file:
  viquad_data_v1 = json.load(viquad_file)

In [ ]:
ls_viquad_v1 = []
for paragraphs in viquad_data_v1['data']:
  for context in paragraphs['paragraphs']:
    for qas in context['qas']:
      dic = {}
      dic['title'] = paragraphs['title']
      dic['context'] = context['context']
      dic['id'] = qas['id']
      dic['question'] = qas['question']
      dic['answer_start'] = []
      dic['text'] = []

      for answer in qas['answers']:
        dic['answer_start'].append(answer['answer_start'])
        dic['text'].append(answer['text'])

      ls_viquad_v1.append(dic)

df_viquad_v1 = pd.DataFrame(ls_viquad_v1)
len(df_viquad_v1)

2210

In [ ]:
df_viquad_v1

,title,context,id,question,answer_start,text
0,Đức,"Năm 1871, Đức trở thành một quốc gia dân tộc k...",uit_01__00061_0_1,Cộng hòa Weimar chính thức thay thế đế quốc Đứ...,"[124, 124, 124, 124]",[Chiến tranh thế giới thứ nhất và Cách mạng Đứ...
1,Đức,"Năm 1871, Đức trở thành một quốc gia dân tộc k...",uit_01__00061_0_2,Nước Đức hiện nay sự hợp thành của hai nước nà...,"[432, 432, 432, 432]",[Cộng hòa Liên bang Đức và Cộng hòa Dân chủ Đứ...
2,Đức,"Năm 1871, Đức trở thành một quốc gia dân tộc k...",uit_01__00061_0_3,Hậu quả nào đã xảy ra khi tại Đức chế độ độc t...,"[311, 311, 311, 311]",[Chiến tranh thế giới thứ hai và một nạn diệt ...
3,Đức,"Năm 1871, Đức trở thành một quốc gia dân tộc k...",uit_01__00061_0_4,Cộng hòa Liên bang Đức và Cộng hòa Dân chủ Đức...,"[480, 484, 480, 480]","[Năm 1990, 1990, Năm 1990, Năm 1990]"
4,Đức,"Trong thế kỷ XXI, Đức là một đại cường quốc và...",uit_01__00061_1_1,"Vào năm 2015, cả hai lĩnh vực nào Đức đều xếp ...","[223, 223, 223, 223]","[xuất khẩu và nhập khẩu, xuất khẩu và nhập khẩ..."
...,...,...,...,...,...,...
2205,Wikipedia,Các yêu cầu trang được gửi cho tầng máy Squid ...,uit_01__00059_13_1,Nếu những yêu cầu mà bộ nhớ Squid không thể th...,"[102, 102, 102, 102]",[được gửi qua các máy chủ cân bằng tải (load-b...
2206,Wikipedia,Các yêu cầu trang được gửi cho tầng máy Squid ...,uit_01__00059_13_2,Cần làm gì để tăng tốc độ trả lời cho người ch...,"[431, 431, 431, 431]",[các trang được kết xuất cho người chưa đăng n...
2207,Wikipedia,Các yêu cầu trang được gửi cho tầng máy Squid ...,uit_01__00059_13_3,Các máy chủ cân bằng tải gửi yêu cầu cho máy c...,"[251, 251, 251, 251]","[kết xuất trang dùng dữ liệu từ CSDL, kết xuất..."
2208,Wikipedia,Các yêu cầu trang được gửi cho tầng máy Squid ...,uit_01__00059_13_4,Để giải quyết các yêu cầu thì máy chủ web có n...,"[304, 288, 304, 304]",[gửi lại những trang được yêu cầu và kết xuất ...


In [ ]:
df_viquad_v1 = df_viquad_v1.drop(['title', 'context','id','answer_start'], axis=1)
df_viquad_v1

,question,text
0,Cộng hòa Weimar chính thức thay thế đế quốc Đứ...,[Chiến tranh thế giới thứ nhất và Cách mạng Đứ...
1,Nước Đức hiện nay sự hợp thành của hai nước nà...,[Cộng hòa Liên bang Đức và Cộng hòa Dân chủ Đứ...
2,Hậu quả nào đã xảy ra khi tại Đức chế độ độc t...,[Chiến tranh thế giới thứ hai và một nạn diệt ...
3,Cộng hòa Liên bang Đức và Cộng hòa Dân chủ Đức...,"[Năm 1990, 1990, Năm 1990, Năm 1990]"
4,"Vào năm 2015, cả hai lĩnh vực nào Đức đều xếp ...","[xuất khẩu và nhập khẩu, xuất khẩu và nhập khẩ..."
...,...,...
2205,Nếu những yêu cầu mà bộ nhớ Squid không thể th...,[được gửi qua các máy chủ cân bằng tải (load-b...
2206,Cần làm gì để tăng tốc độ trả lời cho người ch...,[các trang được kết xuất cho người chưa đăng n...
2207,Các máy chủ cân bằng tải gửi yêu cầu cho máy c...,"[kết xuất trang dùng dữ liệu từ CSDL, kết xuất..."
2208,Để giải quyết các yêu cầu thì máy chủ web có n...,[gửi lại những trang được yêu cầu và kết xuất ...


In [ ]:
def pre_processing(text, vncorenlp: VnCoreNLP):
  phrase_extraction = re.split(r'[\W_] ', text)

  ls = []
  for phrase in phrase_extraction:
    POS_tagging = vncorenlp.pos_tag(phrase)
    ls = ls + POS_tagging

  return ls



In [ ]:
vncorenlp_file = "/content/VnCoreNLP/VnCoreNLP-1.1.1.jar"
vncorenlp = VnCoreNLP(vncorenlp_file) 

In [ ]:
ls = []
for question in df_viquad_v1['question']:
  ls.append(pre_processing(question, vncorenlp))
df_viquad_v1['pre_processing_question'] = ls
df_viquad_v1

,question,text,pre_processing_question
0,Cộng hòa Weimar chính thức thay thế đế quốc Đứ...,[Chiến tranh thế giới thứ nhất và Cách mạng Đứ...,"[[(Cộng_hoà, N), (Weimar, Np), (chính_thức, A)..."
1,Nước Đức hiện nay sự hợp thành của hai nước nà...,[Cộng hòa Liên bang Đức và Cộng hòa Dân chủ Đứ...,"[[(Nước, N), (Đức, Np), (hiện_nay, N), (sự, N)..."
2,Hậu quả nào đã xảy ra khi tại Đức chế độ độc t...,[Chiến tranh thế giới thứ hai và một nạn diệt ...,"[[(Hậu_quả, N), (nào, P), (đã, R), (xảy, V), (..."
3,Cộng hòa Liên bang Đức và Cộng hòa Dân chủ Đức...,"[Năm 1990, 1990, Năm 1990, Năm 1990]","[[(Cộng_hoà, N), (Liên_bang, Np), (Đức, Np), (..."
4,"Vào năm 2015, cả hai lĩnh vực nào Đức đều xếp ...","[xuất khẩu và nhập khẩu, xuất khẩu và nhập khẩ...","[[(Vào, V), (năm, N), (2015, M)], [(cả, P), (h..."
...,...,...,...
2205,Nếu những yêu cầu mà bộ nhớ Squid không thể th...,[được gửi qua các máy chủ cân bằng tải (load-b...,"[[(Nếu, C), (những, L), (yêu_cầu, N), (mà, C),..."
2206,Cần làm gì để tăng tốc độ trả lời cho người ch...,[các trang được kết xuất cho người chưa đăng n...,"[[(Cần, V), (làm, V), (gì, P), (để, E), (tăng,..."
2207,Các máy chủ cân bằng tải gửi yêu cầu cho máy c...,"[kết xuất trang dùng dữ liệu từ CSDL, kết xuất...","[[(Các, L), (máy_chủ, N), (cân_bằng, A), (tải,..."
2208,Để giải quyết các yêu cầu thì máy chủ web có n...,[gửi lại những trang được yêu cầu và kết xuất ...,"[[(Để, E), (giải_quyết, V), (các, L), (yêu_cầu..."


In [ ]:
df_viquad_v1.head(5)

,question,text,pre_processing_question
0,Cộng hòa Weimar chính thức thay thế đế quốc Đứ...,[Chiến tranh thế giới thứ nhất và Cách mạng Đứ...,"[[(Cộng_hoà, N), (Weimar, Np), (chính_thức, A)..."
1,Nước Đức hiện nay sự hợp thành của hai nước nà...,[Cộng hòa Liên bang Đức và Cộng hòa Dân chủ Đứ...,"[[(Nước, N), (Đức, Np), (hiện_nay, N), (sự, N)..."
2,Hậu quả nào đã xảy ra khi tại Đức chế độ độc t...,[Chiến tranh thế giới thứ hai và một nạn diệt ...,"[[(Hậu_quả, N), (nào, P), (đã, R), (xảy, V), (..."
3,Cộng hòa Liên bang Đức và Cộng hòa Dân chủ Đức...,"[Năm 1990, 1990, Năm 1990, Năm 1990]","[[(Cộng_hoà, N), (Liên_bang, Np), (Đức, Np), (..."
4,"Vào năm 2015, cả hai lĩnh vực nào Đức đều xếp ...","[xuất khẩu và nhập khẩu, xuất khẩu và nhập khẩ...","[[(Vào, V), (năm, N), (2015, M)], [(cả, P), (h..."


## Candidate identification and combination

In [ ]:
def is_keyword(pos_word: list) -> bool:
  if 'N' in pos_word[1] or 'A' in pos_word[1] or 'V' in pos_word[1] or 'M' in pos_word[1]:
    return True
  return False

def tuple_str(word1: str, word2: str) -> tuple:
  if word1 > word2:
    return (word2, word1)
  return (word1, word2)

def do_exist_word_pair(word1: str, word2: str, df):
  value = len(df.loc[(df['c1'] == word1) & (df['c2'] == word2)]) 
  if value > 0:
    return True
  return False

## Processing $check$ function for the dataset

Result: csv file containing $count(c_1c_2)$, $count(Nonc_1c_2)$ and $check$ function value of keywords' pairs

$check=\frac{count(c_1c_2)}{count(Nonc_1c_2) + count(c_1c_2)}$

In [ ]:
comb_kw_df = pd.DataFrame(columns=['c1', 'c2', 'count(c1c2)', 'count(Nonc1c2)'])

for question_idx, question in enumerate(tqdm(df_viquad_v1['pre_processing_question'], desc='Processing Progress')):
  sentence = question[0] # because this question contain 1 sentence only
  ls_keyword = []
  ls_c1c2_pair = [] # contain pairs of keywords next to each other

  for idx in range(len(sentence)-1):
    word_1 = sentence[idx]
    word_2 = sentence[idx+1]
    w1 = word_1[0] # get the element contain word only
    w2 = word_2[0]

    if idx == len(sentence)-2 and is_keyword(word_2):
      ls_keyword.append(w2)

    if is_keyword(word_1):
      ls_keyword.append(w1)

      if is_keyword(word_2) and tuple_str(w1, w2) not in ls_c1c2_pair:
        ls_c1c2_pair.append(tuple_str(w1, w2))
        if do_exist_word_pair(w1, w2, comb_kw_df):
          # increase count(w1w2) by 1
          comb_kw_df.loc[(comb_kw_df['c1'] == w1) & (comb_kw_df['c2'] == w2), 'count(c1c2)'] = comb_kw_df.loc[(comb_kw_df['c1'] == w1) & (comb_kw_df['c2'] == w2), 'count(c1c2)'] + 1
        else:
          comb_kw_df.loc[-1] = [w1, w2, 1, 0]
          comb_kw_df.index = comb_kw_df.index + 1

  ls_keyword.sort()
  ls_keyword_pairs = list(itertools.combinations(ls_keyword, 2))
  ls_Nonc1c2_pair = [x for x in ls_keyword_pairs if x not in ls_c1c2_pair] # contain pairs of keywords which are not next to each other in the sentence

  for keyword_pair in ls_Nonc1c2_pair:
    w1 = keyword_pair[0]
    w2 = keyword_pair[1]

    # increase count(Nonw1w2) by 1
    if do_exist_word_pair(w1, w2, comb_kw_df):
      comb_kw_df.loc[(comb_kw_df['c1'] == w1) & (comb_kw_df['c2'] == w2), 'count(Nonc1c2)'] = comb_kw_df.loc[(comb_kw_df['c1'] == w1) & (comb_kw_df['c2'] == w2), 'count(Nonc1c2)'] + 1
    else:
      comb_kw_df.loc[-1] = [w1, w2, 0, 1]
      comb_kw_df.index = comb_kw_df.index + 1
    
    # increase count(Nonw2w1) by 1
    if do_exist_word_pair(w2, w1, comb_kw_df):
      comb_kw_df.loc[(comb_kw_df['c1'] == w2) & (comb_kw_df['c2'] == w1), 'count(Nonc1c2)'] = comb_kw_df.loc[(comb_kw_df['c1'] == w2) & (comb_kw_df['c2'] == w1), 'count(Nonc1c2)'] + 1
    else:
      comb_kw_df.loc[-1] = [w2, w1, 0, 1]
      comb_kw_df.index = comb_kw_df.index + 1

  # temporarily save the duration 
  if question_idx % 250 == 249:
    comb_kw_df.to_csv('/content/drive/MyDrive/Colab Notebooks/comb_kw_df_testViQuAD_in_process.csv', encoding='utf-8', index=False)
    txt_file = open('/content/drive/MyDrive/Colab Notebooks/comb_kw_df_testViQuAD_idx.txt', "w")
    txt_file.write(str(question_idx))
    txt_file.close()
    

# comb_kw_df.drop(comb_kw_df[(comb_kw_df['count(c1c2)'] == 0) & (comb_kw_df['count(Nonc1c2)'] == 0)].index, inplace=True)
comb_kw_df.sort_values(by=['c1', 'c2'], ignore_index=True)

# calculate check value for keyword pairs
ls = []
for index, row in comb_kw_df.iterrows():
  check = row['count(c1c2)'] / (row['count(Nonc1c2)'] + row['count(c1c2)'])
  ls.append(check)
comb_kw_df['check'] = ls

# save data
comb_kw_df.to_csv('/content/drive/MyDrive/Colab Notebooks/comb_kw_df_testViQuAD_May24_upgrade_v2.csv', encoding='utf-8', index=False)

# remove temporary files
if os.path.exists('/content/drive/MyDrive/Colab Notebooks/comb_kw_df_testViQuAD_in_process.csv'):
  os.remove('/content/drive/MyDrive/Colab Notebooks/comb_kw_df_testViQuAD_in_process.csv')
if os.path.exists('/content/drive/MyDrive/Colab Notebooks/comb_kw_df_testViQuAD_idx.txt'):
  os.remove('/content/drive/MyDrive/Colab Notebooks/comb_kw_df_testViQuAD_idx.txt')

Processing Progress: 100%|██████████| 2210/2210 [18:30<00:00,  1.99it/s]


In [ ]:
candidate_key_phrase_df = comb_kw_df.loc[comb_kw_df['check'] > 0.5]
candidate_key_phrase_df

,c1,c2,count(c1c2),count(Nonc1c2),check
62376,Cộng_hoà,Weimar,1,0,1.0
62375,Weimar,chính_thức,1,0,1.0
62374,chính_thức,thay_thế,1,0,1.0
62373,thay_thế,đế_quốc,1,0,1.0
62372,đế_quốc,Đức,1,0,1.0
...,...,...,...,...,...
52,web,có,1,0,1.0
29,nhóm,máy_chủ,1,0,1.0
28,máy_chủ,lớn,1,0,1.0
27,lớn,giải_quyết,1,0,1.0


## Using

In [ ]:
def check_value(keyword_0, keyword_1, df_check):
  try:
    check = df_check.loc[(df_check['c1'] == keyword_0) & (df_check['c2'] == keyword_1), 'check']
    return check.values[0]
  except:
    return -1


def extract_key_phrase(question: str, vncorenlp: VnCoreNLP, df_check):
  """
    extract key phrase from question

    :param question: input question
    :param vncorenlp: VnCoreNLP object to handle POS tagging
    :param df_check: DataFrame contain check value of key word pairs
    :return: list of key phrase
  """
  phrase_extraction = re.split(r'[\W_] ', question)
  ls = []

  for phrase in phrase_extraction:
    POS_tagging = vncorenlp.pos_tag(phrase)
    ls = ls + POS_tagging

  ls_keyphrase = []
  for sentence in ls:
    pass_word = False # de pass word tiep theo neu xac dinh duoc key-phrase
    for idx in range(len(sentence)-1):
      word_1 = sentence[idx]
      word_2 = sentence[idx+1]
      w1 = word_1[0]
      w2 = word_2[0]
      if pass_word:
        pass_word = False
        continue
      if is_keyword(word_1):
        if is_keyword(word_2):
          if check_value(w1, w2, df_check) >= 0.5: # la 1 key-phrase
            key_phrase = w1 + ' ' + w2
            key_phrase = re.sub('_', ' ', key_phrase)
            ls_keyphrase.append(key_phrase)
            pass_word = True
          else:
            w1 = re.sub('_', ' ', w1)
            ls_keyphrase.append(w1)
            if idx == len(sentence)- 2:
              w2 = re.sub('_', ' ', w2)
              ls_keyphrase.append(w2)
        else:
          w1 = re.sub('_', ' ', w1)
          ls_keyphrase.append(w1)

  return ls_keyphrase

In [ ]:
vncorenlp_file = "/content/VnCoreNLP/VnCoreNLP-1.1.1.jar"
vncorenlp = VnCoreNLP(vncorenlp_file)
check_keypair_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/comb_kw_df_testViQuAD_May24_upgrade_v2.csv', encoding='utf-8')

In [ ]:
for i in range(45, 65):
  print(df_viquad_v1['question'][i])
  print(extract_key_phrase(df_viquad_v1['question'][i], vncorenlp, check_keypair_df))
  print('')

Các vùng đất nào đã có sự phát triển mạnh mẽ của thương nghiệp?
['vùng đất', 'có', 'sự', 'phát triển', 'mạnh mẽ', 'thương nghiệp']

Việc truyền bá kiến thức đến đại chúng đã được đặt cơ sở bởi sự kiện nào?
['Việc', 'truyền bá', 'kiến thức', 'đại chúng', 'được', 'đặt', 'cơ sở', 'sự kiện']

Các nghệ sĩ nổi bật nào đã được sản sinh ra từ các trung tâm nghệ thuật và văn hóa trên lãnh thổ Đức?
['nghệ sĩ', 'nổi bật', 'được', 'sản sinh', 'trung tâm', 'nghệ thuật', 'văn hoá', 'lãnh thổ', 'Đức']

Cuộc cải cách nào đã được bắt đầu từ việc tu sĩ Martin Luther ban hành 95 luận đề?
['Cuộc', 'cải cách', 'được', 'bắt đầu', 'việc', 'tu sĩ', 'Martin Luther', 'ban hành', 'luận đề']

Một nguyên tắc nào đã hình thành về đức tin của thần dân và cả vương công?
['nguyên tắc', 'hình thành', 'đức tin', 'thần dân', 'vương', 'công']

Đưc stin nào đã bị xem như là dị giáo?
['Đưc', 'stin', 'bị', 'xem', 'là', 'dị giáo']

Một số nơi của Đức, tỉ lệ suy giảm dân số đạt cao nhất là bao nhiêu trong Chiến tranh Ba mươi N

# Performance

## Get random 200 question

In [ ]:
df_samle_200 = df_viquad_v1.sample(n = 200)
df_samle_200.reset_index(inplace = True, drop = True)
df_samle_200 = df_samle_200.drop(['text'], axis=1)
df_samle_200

,question
0,Nhân khẩu nội thành Trường An thời đường theo ...
1,Ai là người đã thiết kế nên nhà kính công nươn...
2,Hợp chất nào tạo nên màng tế bào thời kỳ sơ khai?
3,Mục đích thành lập dự án là gì?
4,Chiều dài của sông Rhein tại lãnh thổ Đức là b...
...,...
195,Ai là người đã thắng cử chức vụ thủ tướng tron...
196,Sự suy sụp sức khỏe của Hawking trùng hợp lúc ...
197,Quân An Lộc Sơn khi vào thành Trường An đã làm...
198,Lí do nào khiến phụ thân của Edward chuẩn bị c...


In [ ]:
ls_1 = []
ls_2 = []
for question in df_samle_200['question']:
  ls_1.append(extract_key_phrase(question, vncorenlp, check_keypair_df))
  ls_2.append('')
df_samle_200['auto key-phrase extraction'] = ls_1
df_samle_200['manual key-phrase extraction'] = ls_2
df_samle_200

,question,auto key-phrase extraction,manual key-phrase extraction
0,Nhân khẩu nội thành Trường An thời đường theo ...,"[Nhân khẩu, nội thành, Trường, An, thời, đường...",
1,Ai là người đã thiết kế nên nhà kính công nươn...,"[là, người, thiết kế, nhà kính, công nương, xứ...",
2,Hợp chất nào tạo nên màng tế bào thời kỳ sơ khai?,"[Hợp chất, tạo, nên, màng tế bào, thời kỳ, sơ ...",
3,Mục đích thành lập dự án là gì?,"[Mục đích, thành lập, dự án, là]",
4,Chiều dài của sông Rhein tại lãnh thổ Đức là b...,"[Chiều, dài, sông, Rhein, lãnh thổ, Đức, là]",
...,...,...,...
195,Ai là người đã thắng cử chức vụ thủ tướng tron...,"[là, người, thắng cử, chức vụ, thủ tướng, cuộc...",
196,Sự suy sụp sức khỏe của Hawking trùng hợp lúc ...,"[Sự, suy sụp, sức khoẻ, Hawking, trùng hợp, lú...",
197,Quân An Lộc Sơn khi vào thành Trường An đã làm...,"[Quân, An Lộc Sơn, khi, vào, thành, Trường, An...",
198,Lí do nào khiến phụ thân của Edward chuẩn bị c...,"[Lí do, khiến, phụ thân, Edward, chuẩn bị, ông...",


In [ ]:
df_samle_200.to_excel(r'/content/drive/MyDrive/Colab Notebooks/df_samle_200_Jun01.xlsx', index = False)

## Evaluate

In [ ]:
test_200 = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/df_samle_200_Jun01.xlsx')
test_200

,question,manual key-phrase extraction,auto key-phrase extraction
0,Nhân khẩu nội thành Trường An thời đường theo ...,"Nhân khẩu, nội thành, Trường An, thời đường, ư...","['Nhân khẩu', 'nội thành', 'Trường', 'An', 'th..."
1,Ai là người đã thiết kế nên nhà kính công nươn...,"người, thiết kế, nhà kính, công nương, xứ Wales","['là', 'người', 'thiết kế', 'nhà kính', 'công ..."
2,Hợp chất nào tạo nên màng tế bào thời kỳ sơ khai?,"Hợp chất, tạo nên, màng tế bào, thời kỳ sơ khai","['Hợp chất', 'tạo', 'nên', 'màng tế bào', 'thờ..."
3,Mục đích thành lập dự án là gì?,"Mục đích, thành lập, dự án","['Mục đích', 'thành lập', 'dự án', 'là']"
4,Chiều dài của sông Rhein tại lãnh thổ Đức là b...,"Chiều dài, sông Rhein, lãnh thổ Đức","['Chiều', 'dài', 'sông', 'Rhein', 'lãnh thổ', ..."
...,...,...,...
195,Ai là người đã thắng cử chức vụ thủ tướng tron...,"người, thắng cử, chức vụ thủ tướng, cuộc bầu c...","['là', 'người', 'thắng cử', 'chức vụ', 'thủ tư..."
196,Sự suy sụp sức khỏe của Hawking trùng hợp lúc ...,"Sự suy sụp, sức khỏe, Hawkingm, trùng hợp, lúc...","['Sự', 'suy sụp', 'sức khoẻ', 'Hawking', 'trùn..."
197,Quân An Lộc Sơn khi vào thành Trường An đã làm...,"Quân An Lộc Sơn, khi, vào, thành Trương An, làm","['Quân', 'An Lộc Sơn', 'khi', 'vào', 'thành', ..."
198,Lí do nào khiến phụ thân của Edward chuẩn bị c...,"Lí do, phụ thân, Edward, chuẩn bị, ông, mối hô...","['Lí do', 'khiến', 'phụ thân', 'Edward', 'chuẩ..."


In [ ]:
len(test_200)

200

In [ ]:

auto_kp_raw = test_200['auto key-phrase extraction'][0]
auto_kp_raw = re.sub('\[', '', auto_kp_raw)
auto_kp_raw = re.sub('\]', '', auto_kp_raw)
auto_kp_raw = re.sub('\'', '', auto_kp_raw)
auto_key_phrase = auto_kp_raw.split(', ')
auto_key_phrase

manual_kp_raw = test_200['manual key-phrase extraction'][0]
manual_key_phrase = manual_kp_raw.split(', ')
manual_key_phrase

['Nhân khẩu',
 'nội thành',
 'Trường An',
 'thời đường',
 'ước tính',
 'khoảng',
 'bao nhiêu',
 'người']

In [ ]:
def pre_processing_sample(test_200, idx):
  if idx < len(test_200):
    auto_kp_raw = test_200['auto key-phrase extraction'][idx]
    auto_kp_raw = re.sub('\[', '', auto_kp_raw)
    auto_kp_raw = re.sub('\]', '', auto_kp_raw)
    auto_kp_raw = re.sub('\'', '', auto_kp_raw)
    auto_key_phrase = auto_kp_raw.split(', ')

    manual_kp_raw = test_200['manual key-phrase extraction'][idx]
    manual_key_phrase = manual_kp_raw.split(', ')
    return auto_key_phrase, manual_key_phrase

  return None

In [ ]:
# Source: https://machinelearningcoban.com/2017/08/31/evaluation/

def precision(pred_ls: list, true_ls: list):
  TP = 0
  FP = 0
  for element in pred_ls:
    if element in true_ls:
      TP = TP + 1

  precision_value = TP/len(pred_ls)
  return precision_value

def recall(pred_ls: list, true_ls: list):
  TP = 0
  FN = 0
  for element in pred_ls:
    if element in true_ls:
      TP = TP + 1
  for element in true_ls:
    if element not in pred_ls:
      FN = FN + 1

  recall_value = TP/(TP+FN)    
  return recall_value

def F1_score(pred_ls: list, true_ls: list):
  precision_value = precision(pred_ls, true_ls)
  recall_value = recall(pred_ls, true_ls)
  
  F1_score_value = 0
  if precision_value + recall_value > 0:
    F1_score_value = 2 * precision_value * recall_value / (precision_value + recall_value)
  return F1_score_value

auto_key_phrase, manual_key_phrase = pre_processing_sample(test_200, 0)
F1_score(auto_key_phrase, manual_key_phrase)

0.5263157894736842

In [ ]:
auto_key_phrase, manual_key_phrase = pre_processing_sample(test_200, 4)
print(auto_key_phrase)
print(manual_key_phrase)

['Chiều', 'dài', 'sông', 'Rhein', 'lãnh thổ', 'Đức', 'là']
['Chiều dài', 'sông Rhein', 'lãnh thổ Đức']


In [ ]:
sum = [0, 0, 0]

for i in range(len(test_200)):
  try:
    auto_key_phrase, manual_key_phrase = pre_processing_sample(test_200, i)
    sum[0] = sum[0] + precision(auto_key_phrase, manual_key_phrase)
    sum[1] = sum[1] + recall(auto_key_phrase, manual_key_phrase)
    sum[2] = sum[2] + F1_score(auto_key_phrase, manual_key_phrase)
  except ZeroDivisionError:
    print(i)
print("Average Precision: " + str(sum[0]/len(test_200)))
print("Average Recall: " + str(sum[1]/len(test_200)))
print("Average F1_score: " + str(sum[2]/len(test_200)))

Average Precision: 0.4416290792540793
Average Recall: 0.5806272338772339
Average F1_score: 0.4973031584609186


In [ ]:
vncorenlp_file = "/content/VnCoreNLP/VnCoreNLP-1.1.1.jar"
vncorenlp = VnCoreNLP(vncorenlp_file)
check_keypair_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/comb_kw_df_testViQuAD_May24_upgrade_v2.csv', encoding='utf-8')

In [ ]:
question = 'Nghệ thuật tạo hình không thể tranh luận có niên đại lâu đời nhất từng phát hiện là gì?'
phrase_extraction = re.split(r'[\W_] ', question)
ls = []

for phrase in phrase_extraction:
  POS_tagging = vncorenlp.pos_tag(phrase)
  ls = ls + POS_tagging
ls

[[('Nghệ_thuật', 'N'),
  ('tạo_hình', 'V'),
  ('không_thể', 'R'),
  ('tranh_luận', 'V'),
  ('có', 'V'),
  ('niên_đại', 'N'),
  ('lâu_đời', 'A'),
  ('nhất', 'R'),
  ('từng', 'R'),
  ('phát_hiện', 'V'),
  ('là', 'V'),
  ('gì', 'P'),
  ('?', 'CH')]]

In [ ]:
def check_key_phrase_by_rule(word_1, word_2, word_3):
  if word_1[1] == 'N':
    selected_types = ['M', 'Np', 'A']
    if word_2[1] in selected_types:
      return True
    if word_2[1] == 'N':
      if word_3 is None:
        return True
      else:
        if word_3[1] not in selected_types:
          return True

  special_word_ls = ['được', 'bị', 'sự']
  if word_1[0] in special_word_ls and word_2[1] == 'V':
    return True

  if word_1[1] == 'R' and 'không' in word_1[0] and word_2[1] == 'V':
    return True
  
  if word_1[1] == 'A'and word_2[0] == 'nhất':
    return True
  

  return False

def extract_key_phrase_v2(question: str, vncorenlp: VnCoreNLP, df_check):
  """
    extract key phrase from question

    :param question: input question
    :param vncorenlp: VnCoreNLP object to handle POS tagging
    :param df_check: DataFrame contain check value of key word pairs
    :return: list of key phrase
  """
  phrase_extraction = re.split(r'[\W_] ', question)
  ls = []

  for phrase in phrase_extraction:
    POS_tagging = vncorenlp.pos_tag(phrase)
    ls = ls + POS_tagging

  ls_keyphrase = []
  for sentence in ls:
    pass_word = False # de pass word tiep theo neu xac dinh duoc key-phrase
    for idx in range(len(sentence)-1):
      word_1 = sentence[idx]
      word_2 = sentence[idx+1]
      word_3 = sentence[idx+2] if idx+2 < len(sentence) else None
      w1 = word_1[0]
      w2 = word_2[0]
      if pass_word:
        pass_word = False
        continue
      if check_key_phrase_by_rule(word_1, word_2, word_3):
        key_phrase = w1 + ' ' + w2
        key_phrase = re.sub('_', ' ', key_phrase)
        ls_keyphrase.append(key_phrase)
        pass_word = True
      elif is_keyword(word_1):
        if is_keyword(word_2):
          if check_value(w1, w2, df_check) >= 0.5: # la 1 key-phrase
            key_phrase = w1 + ' ' + w2
            key_phrase = re.sub('_', ' ', key_phrase)
            ls_keyphrase.append(key_phrase)
            pass_word = True
          else:
            w1 = re.sub('_', ' ', w1)
            ls_keyphrase.append(w1)
            if idx == len(sentence)- 2:
              w2 = re.sub('_', ' ', w2)
              ls_keyphrase.append(w2)
        else:
          w1 = re.sub('_', ' ', w1)
          ls_keyphrase.append(w1)

  return ls_keyphrase

In [ ]:
question = 'Trong khoảng giai đoạn 1796-1804, những mối bất hòa giữa triều đình với những phần tử chống đối đã gây ra điều gì?'
extract_key_phrase_v2(question, vncorenlp, check_keypair_df)

['khoảng',
 'giai đoạn 1796-1804',
 'mối bất hoà',
 'triều đình',
 'phần tử chống đối',
 'gây',
 'ra',
 'điều']

In [ ]:
sum = [0, 0, 0]

for i in range(len(test_200)):
  auto_key_phrase = extract_key_phrase_v2(test_200['question'][i], vncorenlp, check_keypair_df)
  manual_kp_raw = test_200['manual key-phrase extraction'][i]
  manual_key_phrase = manual_kp_raw.split(', ')
  sum[0] = sum[0] + precision(auto_key_phrase, manual_key_phrase)
  sum[1] = sum[1] + recall(auto_key_phrase, manual_key_phrase)
  sum[2] = sum[2] + F1_score(auto_key_phrase, manual_key_phrase)

print("Average Precision: " + str(sum[0]/len(test_200)))
print("Average Recall: " + str(sum[1]/len(test_200)))
print("Average F1_score: " + str(sum[2]/len(test_200)))

Average Precision: 0.6128522727272723
Average Recall: 0.7105494227994231
Average F1_score: 0.651922783680103


# **IN DEVELOPMENT** ⛔

## Key phrase selection

In [ ]:
def feature_extraction(question, answer, candidate_key_phrase_ls: list):
  key_phrase_ls = []
  for key_phrase in candidate_key_phrase_ls:
    F1_raw = question.count(key_phrase)
    if F1_raw == 0:
      continue
    # F2_raw = 
    # F3_raw = 
    F4_raw = key_phrase in answer and 1 or 0
    F5_raw_raw = question.find(key_phrase)
